# LAB 2 Andrea Mirenda s327644

The way to install any library using a linux terminal aproach is the following:

In [1]:
import os
os.system("pip install pandas")

0

Import with predefined python environment

In [12]:
import pandas as pd
import random
import math
import numpy as np
from icecream import ic
from matplotlib import pyplot as plt
from itertools import accumulate
from itertools import combinations
from tqdm.auto import tqdm
import geopy.distance
from geopy.distance import geodesic

# data import
We use pandas to import and use the data about the cities.
Each city has the coordinates (x and y) which allows us to compute the Euclidean distance between 2 cities.

In [3]:
cities = pd.read_csv('cities/us.csv', header=None, names=['City', 'x', 'y'])
cities

,City,x,y
0,Abilene,32.454514,-99.738147
1,Akron,41.080456,-81.521429
2,Albuquerque,35.105552,-106.647388
3,Alexandria,38.818343,-77.082026
4,Allen,33.107224,-96.674676
...,...,...,...
321,Wichita Falls,33.906699,-98.525854
322,Wilmington,34.209225,-77.885767
323,Winston‐Salem,36.103262,-80.260578
324,Worcester,42.269478,-71.807783


# Helping function

Define a function that extract the coordinates of a given city

In [21]:
def find_coordinates(name_city):
    x_coordinate = cities.loc[cities['City'] == name_city, 'x'].values[0]
    y_coordinate = cities.loc[cities['City'] == name_city, 'y'].values[0]
    return (x_coordinate, y_coordinate)

Define a function that computes the Euclidian distance between two cities

In [5]:
def euclidean_distance_math(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    distanza = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return round(distanza, 3)

Define the distance using the geopy function for more accurate distance computation:

In [6]:
def euclidean_distance(c1, c2):
    x1, y1 = c1
    x2, y2 = c2
    return geopy.distance.geodesic((x1, y1), (x2, y2)).km

Function that return the closest city

In [7]:
def closest_city(coord, cities_df):
    closest_city = None
    min_distance = float('inf')
    for _, row in cities_df.iterrows():
        city_coord = (row['x'], row['y'])
        distance = euclidean_distance(coord, city_coord)
        
        if distance < min_distance:
            min_distance = distance
            closest_city = row['City']
    
    return closest_city, round(min_distance, 2)

Function that return the n closest cities to a given one. This may be helpful in the case the city we want to explore has already been explored.

In [20]:
def closest_cities(coord, cities_df, n):
    distanze_citta = []
    for _, row in cities_df.iterrows():
        city_coord = (row['x'], row['y'])
        distanza = euclidean_distance(coord, city_coord)
        distanze_citta.append((row['City'], round(distanza, 2)))
    
    distanze_citta_ordinate = sorted(distanze_citta, key=lambda x: x[1])
    
    return distanze_citta_ordinate[:n]

# Simple greedy aproach

In this simplified version of the problem we can take account of the cities we have already visited and, until the number of elements inside this set is different from the number of cities of the country considered we simply add the closest city to the last city we visited.

In [22]:
# select random city as a starting point
starting_city = cities.sample(n=1)['City'].values[0]

# mantain a set of cities already visited:
visited = set()
distance = 0 #euclidean distance computed
num_visited = 0
num_cities = cities.shape[0]
current_city = find_coordinates(starting_city) # current city esplored
print("numero di città", num_cities)
found = True
num_next_cities = int(num_cities/5)

# while i did not visited all the cities i keep on visiting:
while num_visited<num_cities:
    # if we didn't find the closest one and we did not see all the cities yet
    # we increase the number of close cities to consider.
    if not found:
        num_next_cities = num_cities#increase the number of close cities to consider
    next_city = None
    found = False 
    #take the list of the first num_next_cities cities:
    
    closest_ones = closest_cities(current_city, cities, num_next_cities)
    for city in closest_ones:
        city = city[0]
        if city not in visited:
            found = True
            coordinate_of_next = find_coordinates(city)
            distance += euclidean_distance(current_city, coordinate_of_next)
            current_city = find_coordinates(city)
            num_visited += 1
            visited.add(city)
            break
    #at the end we must end up to the first position:
    if num_visited==num_cities:
        #compute the distance to go from current city to the first one:
        distance += euclidean_distance(current_city, find_coordinates(starting_city))

#we visited all the cities with cost distance:
print("After visiting all the", len(cities), "cities the cost was: ", round(distance, 3), "Km")    
    

numero di città 326
After visiting all the 326 cities the cost was:  48383.32 Km


# improved aproach
As a first improvement we may decide to avoid computing each time find_coordinates, by saving into a dict the link between a city and his cordinates.

We may also decide avoiding to consider at each iterations all the cities, but considering anly the ones we didn't selected yet.
However these changes will improve only the computation, but not the final cost as number of kilometers.

We first define the matrix of distances between cities:

In [23]:
DIST_MATRIX = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.x, c1.y), (c2.x, c2.y)
    ).km
DIST_MATRIX

array([[   0.        , 1881.53104786,  704.042025  , ..., 1835.22890216,
        2687.41674749, 2484.63529263],
       [1881.53104786,    0.        , 2292.45751903, ...,  563.30186495,
         819.10343728,  643.80788986],
       [ 704.042025  , 2292.45751903,    0.        , ..., 2386.19222301,
        3110.18513959, 2931.95568434],
       ...,
       [1835.22890216,  563.30186495, 2386.19222301, ...,    0.        ,
        1000.02847269,  774.02576714],
       [2687.41674749,  819.10343728, 3110.18513959, ..., 1000.02847269,
           0.        ,  226.01477872],
       [2484.63529263,  643.80788986, 2931.95568434, ...,  774.02576714,
         226.01477872,    0.        ]])

We define the dictionary that contains the link between the city and the coordinates:

In [24]:
city_coordinates = cities.set_index('City')[['x', 'y']].apply(tuple, axis=1).to_dict()

We can now change the "closest_cicties" method:

In [25]:
#create a mapping between index and name of city:
city_to_index = {city: idx for idx, city in enumerate(cities['City'])}

def closest_cities(city_name, cities_df, n):
    city_idx = city_to_index.get(city_name)
    # distances to the city:
    distances = DIST_MATRIX[city_idx]
    # order escluding the same city
    nearest_indices = np.argsort(distances)[1:n+1]
    # get the name of the cities:
    nearest_cities = [(cities.iloc[i]['City'], distances[i]) for i in nearest_indices]
    return nearest_cities

We can also change the find coordinates method:

In [26]:
def find_coordinates(name_city):
    return city_coordinates.get(name_city)

In [27]:
# select random city as a starting point
starting_city = cities.sample(n=1)['City'].values[0]

# mantain a set of cities already visited:
visited = set()
distance = 0 #euclidean distance computed
num_visited = 0
num_cities = cities.shape[0]
current_city = find_coordinates(starting_city) # current city esplored
name_current=starting_city
print("numero di città", num_cities)
found = True
num_next_cities = int(num_cities/5)

# while i did not visited all the cities i keep on visiting:
while num_visited<num_cities:
    # if we didn't find the closest one and we did not see all the cities yet
    # we increase the number of close cities to consider.
    if not found:
        num_next_cities = num_cities#increase the number of close cities to consider
    next_city = None
    found = False 
    #take the list of the first num_next_cities cities:
    
    closest_ones = closest_cities(name_current, cities, num_next_cities)
    for city in closest_ones:
        city = city[0]
        if city not in visited:
            found = True
            coordinate_of_next = find_coordinates(city)
            distance += euclidean_distance(current_city, coordinate_of_next)
            current_city = find_coordinates(city)
            name_current = city
            num_visited += 1
            visited.add(city)
            break
    #at the end we must end up to the first position:
    if num_visited==num_cities:
        #compute the distance to go from current city to the first one:
        distance += euclidean_distance(current_city, find_coordinates(starting_city))

#we visited all the cities with cost distance:
print("After visiting all the", len(cities), "cities the cost was: ", round(distance, 3), "Km")    
    

numero di città 326
After visiting all the 326 cities the cost was:  48866.871 Km


# Hill climbing

We can now consider using the hill climbing approach. the representation could be something like a list of cities visited.

As a fitness function we may decide to use (number of covered cities, total distance).